In [1]:
import pandas as pd
import numpy as np
import findspark
findspark.init('C:\spark\spark-3.1.2-bin-hadoop3.2')
import pyspark

from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark import SparkConf
from pyspark.sql.functions import *
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [51]:
students = spark.read.option("inferschema","true").option("header","true").csv("stud.csv")
grades = spark.read.option("inferschema","true").option("header","true").csv("grades.csv")
students.show(5)
grades.show(5)

+----------+------------+----------------+----------+
|Student_ID|Student_Name|         Subject|      Date|
+----------+------------+----------------+----------+
|         1|       James|    Data Science|2021-11-01|
|         2|        Bing|    Data Science|2021-11-01|
|         3|        Bond|    Data Science|2021-11-03|
|         4|        Mike|    Data Science|2021-11-04|
|         5|         Max|Machine Learning|2021-11-01|
+----------+------------+----------------+----------+
only showing top 5 rows

+----------+-----+
|Student_ID|Grade|
+----------+-----+
|         1|   63|
|         2|   54|
|         3|   77|
|         4|   96|
|         5|   55|
+----------+-----+
only showing top 5 rows



In [46]:
#compute the grade of all students in Big Data Subject and Sort the resulting RDD from highest to lowest grade

In [39]:
rdd1 = students.rdd.filter(lambda row: row["Subject"] == "Data Science")
rdd2 = grades.rdd
rdd3 = rdd1.join(rdd2)
rdd3.collect()

[(2, ('Bing', 54)),
 (4, ('Mike', 96)),
 (1, ('James', 63)),
 (3, ('Bond', 77)),
 (9, ('Rose', 45))]

In [48]:
rdd4 = rdd3.sortBy(lambda x: x[1][1],ascending= False)
rdd4.collect()

[(4, ('Mike', 96)),
 (3, ('Bond', 77)),
 (1, ('James', 63)),
 (2, ('Bing', 54)),
 (9, ('Rose', 45))]

In [49]:
#One Line
students.rdd.filter(lambda row: row["Subject"] == "Data Science").join(grades.rdd).sortBy(lambda x: x[1][1],ascending= False).collect()

[(4, ('Mike', 96)),
 (3, ('Bond', 77)),
 (1, ('James', 63)),
 (2, ('Bing', 54)),
 (9, ('Rose', 45))]

In [50]:
#Students in each subject descending order
students.rdd.map(lambda y: (y["Subject"], 1)).reduceByKey(lambda x,y: x + y).sortBy(lambda x:x[1], ascending= False).collect()

[('Data Science', 6), ('Machine Learning', 4)]

In [52]:
#Students in each subject on 2021-11-01 descending order
students.rdd.filter(lambda row: row['Date'] == '2021-11-01').map(lambda y: (y["Subject"], 1)).reduceByKey(lambda x,y: x + y).sortBy(lambda x:x[1], ascending= False).collect()

[('Data Science', 2), ('Machine Learning', 1)]